In [ ]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord, GCRS
from astropy import units as u
from astropy.time import Time
import numpy as np
from datetime import datetime, timedelta

start_time = datetime(2025, 2, 21, 12, 0, 0)
current_time = start_time

vla = EarthLocation.of_site('vla')
source = SkyCoord(ra=10*u.hourangle, dec=-30*u.deg, frame='icrs')

antenna_1_pos = np.array([0,0,0])
antenna_2_pos = np.array([10,200,10])

# ENU coordinates
baseline = antenna_2_pos - antenna_1_pos

# convert to x,y,z coordinates where z points to the NCP
# we need the latitude of the site
# y will point locally eastward
# z aligns with NCP
# think of projection of NU axes onto z.
# x is y X z
latitude = vla.lat
transform_enu_xyz = np.array([
    [0, -np.sin(latitude), np.cos(latitude)], 
    [1, 0, 0], 
    [0, np.cos(latitude), np.sin(latitude)]
])
baseline_xyz = np.matmul(transform_enu_xyz, baseline)

obs_frequency = 1_000_000
obs_wavelength = 3.0 * 10**8 / obs_frequency 

In [ ]:
import matplotlib.pyplot as plt

uvs = []

for i in range(6000):
    current_time = current_time + timedelta(minutes=1)
    obs_time = Time(current_time.isoformat(), location=vla)
    obs_local_sidereal_time = obs_time.sidereal_time(kind='mean')
    hour_angle = obs_local_sidereal_time - source.ra
    
    # taken from slides
    # u points eastward
    # w tracks source
    # v is the remaining x-product
    transform_xyz_uvw = np.array([
    [np.sin(hour_angle), np.cos(hour_angle), 0],
    [-np.sin(source.dec) * np.cos(hour_angle), np.sin(source.dec) * np.sin(hour_angle), np.cos(source.dec)],
    [np.cos(hour_angle) * np.cos(source.dec), np.sin(hour_angle) * np.cos(source.dec), np.sin(source.dec)]    
])
    baseline_uvw = np.matmul(transform_xyz_uvw, baseline_xyz) / obs_wavelength
    uvs.append(baseline_uvw[:2])

uvs = np.array(uvs)

plt.scatter(uvs[:,0], uvs[:,1], color='blue')

plt.show()


In [ ]:
source.ra 
10 * 15